<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/omake08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML omake08

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

----
## 外食年間支出金額データの K-means クラスタリング
----

授業の notebook でも扱っている都道府県ごとの外食への年間支出金額のデータをクラスタリングして分析してみましょう．



ここで使うのは，独立行政法人統計センターが作成している [教育用標準データセット(SSDSE)](https://www.nstac.go.jp/use/literacy/ssdse/) の中の「SSDSE-家計消費」というデータです．「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files//kaisetsu-C-2022.pdf

この notebook の作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

#### データの入手と前処理

SSDSEのサイトから CSV ファイルを入手して，扱いやすいように加工します．



In [ ]:
URL = 'https://www.nstac.go.jp/sys/files//SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換

# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df.iloc[:, :3]
city = city.drop(1, axis=0)
city['n'] = np.arange(47, dtype=int)
city = city.set_index('n')

print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df = df.drop(['Prefecture', 'City'], axis=1)
df = df.set_index('SSDSE-C-2022')

df = df.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df = df.astype('float32') # 値を浮動小数点型に変換

print()
df.head(5)

ここでは，各種の「外食」での支出金額のみを特徴として取り出します．13個の特徴量があります．

In [ ]:
# 一人あたりの外食支出金額のデータを作成
df_feature = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df.columns:
    if c.startswith('LB12') and c != 'LB12':
        print(c, code[c])
        columns[c] = code[c]
        df_feature[c] = df[c]

# 列名をコードから項目名に変更
df_feature = df_feature.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df_feature = df_feature.divide(df['LA03'], axis=0)

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

df_feature.head(5)

---
#### K-means クラスタリングしてみよう



ここでは，scikit-learn の KMeans クラスを使ってクラスタリングしてみましょう．

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

次のセルを実行すると，クラスタ数を `K` とする `KMeans` のインスタンスを生成します．


In [ ]:
from sklearn.cluster import KMeans
K = 3

# KMeans クラスのインスタンスの生成．クラスタ数は K．verbose=1とすると，学習途中のログを表示させられる
model = KMeans(n_clusters=K, verbose=1)

次に， `KMeans` クラスの学習メソッド に `df_feature` を渡して呼べば， K-means 法の学習が実行されます．`df_feature` は pandas の DataFrame ですが，scikit-learn は NumPy の array と同様に DataFrame も扱えます．

学習の後，クラスタラベルを予測するメソッドを呼べば，どのデータがどのクラスタに分類されたかを出力させることができます．

以下のセルにコードを書いてみましょう．必要ならコードセルを適当に追加してね．

どの都道府県がどのクラスタに入ったか調べたり，セントロイドの値を表示させたりしてみてね．`K` の値も変えてみよう．

クラスタリング結果を確認するときは，次のセルの変数 `y` にクラスタリングで得られたラベルの値を代入してやるとよいでしょう．

In [ ]:
y = np.arange(1, 48, dtype=int)
city['label'] = y
city